In [ ]:
!pip install langchain chromadb PyPDF2 unstructured sentence_transformers
!pip install huggingface_hub transformers
!pip install langchain-community

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
import requests

In [4]:
# Скачивание PDF-файлов по указанным URL
pdf_urls = [
    'https://ptsj.ru/articles/507/507.pdf',
    'https://cyberrus.info/wp-content/uploads/2022/05/27-38-248-22_5.-Vasilyev.pdf',
    'https://www.ria.ee/sites/default/files/documents/2022-11/%D0%9A%D1%80%D0%B0%D1%82%D0%BA%D0%BE%D0%B5-%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D1%81%D1%82%D0%B2%D0%BE-%D0%BF%D0%BE-%D0%BA%D0%B8%D0%B1%D0%B5%D1%80%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D0%B8-%D0%B4%D0%BB%D1%8F-%D0%BE%D1%80%D0%B3%D0%B0%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8.pdf',
    'https://iee.unn.ru/wp-content/uploads/sites/9/2017/02/konspekt-lektsij-po-IB.pdf',
    'https://rrc.ru/themes/rrc/assets/pages/20180627/check%20point-25052018-screen-letter-rus-v3bis.pdf'
]

if not os.path.exists('pdfs'):
    os.makedirs('pdfs')

for idx, url in enumerate(pdf_urls):
    response = requests.get(url)
    with open(f'pdfs/doc_{idx}.pdf', 'wb') as f:
        f.write(response.content)


In [5]:
# Загрузка PDF-файлов и разделение текста на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_documents = []

for idx in range(len(pdf_urls)):
    loader = PyPDFLoader(f'pdfs/doc_{idx}.pdf')
    documents = loader.load()
    all_documents.extend(documents)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(all_documents)

In [6]:
# Создание индекса ChromaDB для поиска
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Chroma.from_documents(split_docs, embeddings)

<ipython-input-6-b02fa52f5625>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acce

In [7]:
# Проверка работы индекса

query = "Расскажи о методах информационной безопасности?"
retrieved_docs_with_scores = vectorstore.similarity_search_with_score(query, k=5)

for idx, (doc, score) in enumerate(retrieved_docs_with_scores):
    print(f"Документ {idx+1} (Сходство: {score}):\n{doc.page_content}\n")

Документ 1 (Сходство: 5.82827091217041):
Служба  система штатных органов управления и организационных  
безопасности  формирований,  предназначенных  для обеспечения  
 безопасности информации.    
Средства   защиты   Средства защиты информации:    
информации   - технические,  криптографические,  программные  и  
   другие  средства,  предназначенные  для  защиты  
   конфиденциальной информации;   
  -  средства реализации средств защиты информации;  
 средства конт роля эффективности защиты информации.  
Стратегическое  1) методология  анализа,  разработки  и  сопровождения  
планирование СИБ  сложных СИБ, определяющая подсистемы, компоненты и  
 способы их соединения, задающая ограничения, при которых  
 система  должна  р азрабатываться  и  функционировать,  
 выбирающая  наиболее  эффективное  сочетание  людей  и  
 технических средств для реализации системы;   
 2)  деятельность, осуществляемую для выработки решений  
 по трансформации начального состояния СИБ к требуемому

Доку

In [8]:
# Загрузка LLM, поддерживающей русский язык, через Hugging Face
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model = model.to("cuda")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length=2048,
    temperature=0.4,
    top_p=0.8,
    repetition_penalty=2.0
)

llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-8-0974523b57af>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
# Запрос к LLM без использования RAG
query = "Какие бывают виды вредоносных программ?"

response = llm(query)

print("Ответ LLM без использования RAG:")
print(response)

<ipython-input-9-6b601f9d145a>:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(query)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `

Ответ LLM без использования RAG:
Какие бывают виды вредоносных программ?
Вредоносные программы - это программа, которая может нанести вред компьютеру или другим устройствам. В основном они создаются для того чтобы красть информацию с компьютера и других устройств (компьютер).
какие есть способы заработать в интернете без вложений??? кроме как на кликах??? подскажите пожалуйста!!!! заранее спасибо!))))) ) очень надо!!!!)))).... а то я уже отчаялась....:(((..а вы что думаете по этому поводу?????..и вообще можно ли зарабатывать деньги сидя дома за компьютером?!или лучше не рисковать своими деньгами!!?)...спасибо всем кто ответит)..заранее спасибки вам!) ************** Спасибо большое тем людям которые мне помогли найти ответы к этим вопросам :-) Я думаю многим будет интересно узнать о том какие существуют методы заработка денег находясь у себя во дворе например если ты живешь один тогда тебе нужно просто выйти из своего подъезда, зайти через калитку домой там где живет твоя семья потом пр

In [10]:
# Реализация RAG
# Создание шаблона промпта на русском языке
prompt_template = """Ответь на вопрос, используя предоставленный контекст. Если информации недостаточно, ответь максимально полно на основе доступных данных.

Контекст:
{context}

Вопрос: {question}
Ответ:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Настройка цепочки с использованием RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT
    }
)

In [12]:
# Обработка случая с отсутствием релевантной информации в базе
threshold = 15
etrieved_docs_with_scores = vectorstore.similarity_search_with_score(query, k=3)
scores = [score for _, score in retrieved_docs_with_scores]

# Проверяем, если все оценки превышают порог (метрика L2 расстояние)
if all(score > threshold for score in scores):
    print('Нет релевантной информации в базе')
    raise SystemExit("Остановлено выполнение ячейки: нет релевантной информации.")

else:
    print('Есть релевантная информация')




# Получение ответа с использованием RAG
result = qa_chain(query)

print("Ответ с использованием RAG:")
print(result['result'])

# Вывод использованных источников
print("\nИспользованные источники:")
for doc in result['source_documents']:
    print(f"Источник: {doc.metadata.get('source')}\nСодержимое: {doc.page_content[:200]}...\n")


Есть релевантная информация


<ipython-input-12-5a93ecf6cb11>:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Ответ с использованием RAG:
Ответь на вопрос, используя предоставленный контекст. Если информации недостаточно, ответь максимально полно на основе доступных данных.

Контекст:
отнести также хорошо известные специалистам продукты программирования как 
"троянские кони", "троянские матрешки", "салями ", "логическая бомба", временная 
бомба, загруженные "атлеты", которые могут использоваться как в положительных 
целях, так и в преступных и иные программы, которые можно отнести к вредоносным. 
В большинстве случаев эти программы используются для хищения ден ег со счетов, 
шпионажа (коммерческого или промышленного), в хулиганских целях, из мести, 
озорства и т.д.  
 
 
 
61

программ, периодическую проверку наличия других возможных следов вирусной 
активности (например, обнаружен ие нарушений целостности программного 
обеспечения), а также входной контроль новых программ перед их использованием (по 
характерным признакам наличия в их теле вирусных образований).  
 Защита от несанкционированн